In [1]:
import numpy as np 
import pandas as pd


import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import warnings
warnings.simplefilter("ignore")

import yfinance as yf

from plotly.subplots import make_subplots


from ta.trend import MACD
from ta.momentum import StochasticOscillator

In [2]:
amd_df = yf.download(tickers='AMD',period='1y',interval='1d')
amd_df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-03,115.160004,116.730003,114.510002,115.820000,115.820000,31860000
2023-07-05,116.099998,117.660004,113.860001,113.949997,113.949997,49501600
2023-07-06,112.070000,113.790001,110.309998,113.480003,113.480003,42342000
2023-07-07,114.160004,115.330002,113.040001,113.169998,113.169998,37135100
2023-07-10,113.000000,113.610001,109.879997,113.580002,113.580002,49330200
...,...,...,...,...,...,...
2024-06-26,160.600006,160.740005,156.000000,157.539993,157.539993,36973400
2024-06-27,157.179993,160.610001,156.809998,159.470001,159.470001,34018200
2024-06-28,160.229996,166.449997,159.410004,162.210007,162.210007,56204600


### Exponential Moving Average

In [3]:
amd_df['MA20'] = amd_df['Adj Close'].ewm(span=20,adjust=False).mean()
amd_df['MA50'] = amd_df['Adj Close'].ewm(span=50,adjust=False).mean()
amd_df

,Open,High,Low,Close,Adj Close,Volume,MA20,MA50
Date,,,,,,,,
2023-07-03,115.160004,116.730003,114.510002,115.820000,115.820000,31860000,115.820000,115.820000
2023-07-05,116.099998,117.660004,113.860001,113.949997,113.949997,49501600,115.641904,115.746666
2023-07-06,112.070000,113.790001,110.309998,113.480003,113.480003,42342000,115.436009,115.657778
2023-07-07,114.160004,115.330002,113.040001,113.169998,113.169998,37135100,115.220198,115.560218
2023-07-10,113.000000,113.610001,109.879997,113.580002,113.580002,49330200,115.063989,115.482562
...,...,...,...,...,...,...,...,...
2024-06-26,160.600006,160.740005,156.000000,157.539993,157.539993,36973400,160.647946,162.090911
2024-06-27,157.179993,160.610001,156.809998,159.470001,159.470001,34018200,160.535761,161.988130
2024-06-28,160.229996,166.449997,159.410004,162.210007,162.210007,56204600,160.695213,161.996831


### Plotting MAs

In [6]:
fig = go.Figure()

candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                       high=amd_df['High'], low=amd_df['Low'],
                       close=amd_df['Close'], name='Candlestick')
ema20 = go.Scatter(x=amd_df.index, y=amd_df['MA20'], 
                   line=dict(color='green', width=1), name="EMA20")

ema50 = go.Scatter(x=amd_df.index, y=amd_df['MA50'], 
                  line=dict(color='orange', width=1), name="EMA50")

fig.add_trace(candle)
fig.add_trace(ema20)
fig.add_trace(ema50)      

fig.update_layout(title="AMD")

fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
                       

### Moving Average Convergence & Divegence (MACD)

In [7]:
def plot_macd_stoch(ticker, time_period, time_int):
    # Download stock data
    stock_df = yf.download(tickers=ticker, period=time_period, interval=time_int)
    
    candle = go.Candlestick(x=stock_df.index, open=stock_df['Open'],
                       high=stock_df['High'], low=stock_df['Low'],
                       close=stock_df['Close'])

    stock_df['MA12'] = stock_df['Adj Close'].ewm(span=12, adjust=False).mean()
    stock_df['MA26'] = stock_df['Adj Close'].ewm(span=26, adjust=False).mean()
  
    macd = MACD(close=stock_df['Close'],
             window_slow=26,
             window_fast=12,
             window_sign=9)

    sto_os = StochasticOscillator(high=stock_df['High'],
                             close=stock_df['Close'],
                             low=stock_df['Low'],
                             window=14, 
                             smooth_window=3)

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    fig.add_trace(candle, row=1, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)
    
    fig.add_trace(go.Bar(x=stock_df.index, 
                     y=macd.macd_diff()
                    ), row=2, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch(),
                         line=dict(color='blue', width=2)
                        ), row=3, col=1)
   
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch_signal(),
                         line=dict(color='orange', width=2)
                        ), row=3, col=1)
    
    fig.add_hline(y=20, line_width=1, line_dash='dash',
                  line_color='red', row=3, col=1)
    fig.add_hline(y=80, line_width=1, line_dash='dash',
                  line_color='green', row=3, col=1)
    
    fig.add_trace(go.Bar(x=stock_df.index,
                         y=stock_df['Volume']),
                         row=4, col=1)
    
    fig.update_layout(title=ticker)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="Stoch", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)

    fig.update_layout(height=900, width=1200,
                      showlegend=False,
                      xaxis_rangeslider_visible=False,
                      xaxis_rangebreaks = [dict(bounds= ['Sat','Mon']),
                                           dict(bounds=[15.5,9.25],pattern='hour')])
    
    fig.show()

In [8]:
plot_macd_stoch("BAJAJFINSV.NS","5d","5m")

[*********************100%%**********************]  1 of 1 completed
